# Importing Dependencies

In [1]:
import os
import json
import time

from dotenv import load_dotenv
from google import genai
from google.genai import types

from typing import List

In [2]:
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [3]:
client = genai.Client(api_key=gemini_api_key)

# Root Features

In [4]:
# # All root features
# with open("../data/root_features.json", "r", encoding="utf-8") as file:
#     root_features = json.load(file)

# Selected root features
with open("../data/selected_features.json", "r", encoding="utf-8") as file:
    root_features = json.load(file)

# Functions

In [5]:
system_prompt = """"
"You are an expert in mobile app development and requirements engineering. 
You excel at decomposing high-level features into detailed sub-features.
"""

In [6]:
def get_level1_prompt(feature: str, feature_description: str, num_features: int = 5) -> str:
    return f"""
**Feature**
```
{feature}: {feature_description}
```
Given the mobile app feature above, please refine it to a list of sub-features.
Ensure that the number of sub-features is {num_features}.
The output should be a list of JSON formatted objects like this:
[{{
"sub_feature": sub_feature,
"description": description
}}]
"""

In [7]:
def get_level2_prompt(feature: str, feature_description: str, super_feature: str, super_feature_description:str, 
                      siblings_features: List[str], num_features: int = 5) -> str:
    return f"""
**Super Feature**
```
super-feature: {super_feature}
description: {super_feature_description}
```
Knowing that the feature "{super_feature}" above is refined into a list of the following features:
```
{siblings_features}
```

Please refine the following to a list of sub-features.
Ensure that the number of sub-features is {num_features}.

**Feature**
```
feature: {feature}
description: {feature_description}
```

The output should be a list of JSON formatted objects like this:
[
    {{
        "sub_feature": sub_feature,
        "description": description
    }}
]
"""

# Level 1 Feature Refinements

In [ ]:
first_level_results = []

for feature in root_features:
    prompt = get_level1_prompt(
        feature["feature"],
        feature["description"]
    )

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        config=types.GenerateContentConfig(system_instruction=system_prompt),
        contents=prompt
    )

    first_level_results.append({
        "feature": feature["feature"],
        "description": feature["description"],
        "sub_features": json.loads((response.text).replace('```', '').replace('json', ''))
    })

    # Minimizar problemas de limite de requisicao por minuto
    time.sleep(5)

# Salvar os resultados em um arquivo JSON
with open("../data/gemini/base_prompt/first_level_subfeatures.json", "w") as f:
    json.dump(first_level_results, f, indent=4)

# Level 2 Feature Refinements

In [ ]:
second_level_results = []

for super_feature in first_level_results:
    for feature in super_feature["sub_features"]:
        prompt = get_level2_prompt(
            feature["sub_feature"], 
            feature["description"],
            super_feature["feature"],
            super_feature["description"],
            [f["sub_feature"] for f in super_feature["sub_features"]]
        )

        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(system_instruction=system_prompt),
            contents=prompt
        )

        second_level_results.append({
            "super_feature": super_feature["feature"],
            "super_feature_description": super_feature["description"],
            "feature": feature["sub_feature"],
            "description": feature["description"],
            "siblings": [f["sub_feature"] for f in super_feature["sub_features"]],
            "sub_features": json.loads((response.text).replace('```', '').replace('json', ''))
        })

        # Minimizar problemas de limite de requisicao por minuto
        time.sleep(5)

# Salvar os resultados em um arquivo JSON
with open("../data/gemini/base_prompt/second_level_subfeatures.json", "w") as f:
    json.dump(second_level_results, f, indent=4)